# Imports and setting up viz

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_routing import *

# Load files
File to evaluate the behaviour of native routing compared to standard.

First working on the same topographic grid (0.5°).

3 files are considered : one from the standard routing simulation, 2 from the native routing simulation (sechiba_history, diag_routing_r)

In [2]:
dir='../../../JZ_simu_outputs/routing_native'

In [3]:
spinupyears=[2000,2001,2002]
# spinupyears=[]

## Format functions

In [4]:
def format_routing_ORC(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter': 'time'})
    ds.attrs['name'] = name
    ds = remove_years_efficiently(ds, spinupyears)

    #edit units and names
    if 'hydrographs' in ds:
        ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
        ds['hydrographs'].attrs['long_name'] = 'River discharge'
    if 'streamr' in ds:
        ds['streamr'].attrs['units'] = 'kg m⁻²'
        ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    if 'slowr' in ds:
        ds['slowr'].attrs['units'] = 'kg m⁻²'
        ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    if 'fastr' in ds:
        ds['fastr'].attrs['units'] = 'kg m⁻²'
        ds['fastr'].attrs['long_name'] = 'Overland reservoir volume'

    if color is not None:
        ds.attrs['plot_color'] = color
    
    if 'coastalflow' in ds and 'riverflow' in ds:
        ds['coast_and_river'] = ds['coastalflow'] + ds['riverflow']
        ds['coast_and_river'].attrs['units'] = 'm³ s⁻¹'
        ds['coast_and_river'].attrs['long_name'] = 'Coastal and river flow'

    return ds

In [5]:
def format_routing_native(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
        'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_riverflow_r' : 'riverflow',
        'routing_coastalflow_r' : 'coastalflow'
        }
    ds = ds.rename(dict)
    ds = ds.isel(lon=slice(2,-2),lat=slice(2,-2))

    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    #edit units and names
    ds['hydrographs'].attrs['units'] = 'm³/s'
    ds['hydrographs'].attrs['long_name'] = 'River discharge'
    return ds

In [6]:
def format_routing_nativeDIAG(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
            
            # for reservoirs
            'lat_domain_landpoints_routing':'lat',
            'lon_domain_landpoints_routing':'lon',
            # for drainage and runoff
            # 'lat_domain_landpoints':'lat',
            # 'lon_domain_landpoints':'lon',

            'routing_stream_reservoir' : 'streamr',
            'routing_fast_reservoir' : 'fastr',
            'routing_slow_reservoir' : 'slowr',
            'routing_drainage' : 'drainage',
            'routing_runoff' : 'runoff',
            'routing_riverflow' : 'riverflow',
            'routing_coastalflow' : 'coastalflow',
            'routing_irrigation' : 'irrigation',}
    #change the variables that exist according to the dictionary
    ds = ds.rename(dict)
    #remove first years to consider a spinup
    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    return(ds)

## No irrigation

In [7]:
#open netcdf files for standard routing
filename = '{}/eval_halfdeg/nat7.1/MO/nat7.1_*_1M_sechiba_history.nc'.format(dir)
name= 'subgrid_halfdeg, no_irr'
std_noirr = format_routing_ORC(filename, name, spinupyears, color='red')
std_noirr

<xarray.Dataset> Size: 5GB
Dimensions:                 (time: 3653, axis_nbounds: 2, lat: 36, lon: 38,
                             solay: 11, veget: 15, nobio: 1)
Coordinates:
  * lat                     (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                     (lon) float32 152B -14.75 -14.25 ... 3.25 3.75
  * solay                   (solay) float32 44B 0.0 0.001955 ... 0.999 2.0
  * veget                   (veget) float32 60B 1.0 2.0 3.0 ... 13.0 14.0 15.0
  * nobio                   (nobio) float32 4B 1.0
    time_centered           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 29kB 2003-01-01T12:00:00 .....
Dimensions without coordinates: axis_nbounds
Data variables: (12/71)
    time_centered_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds     (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    Areas                   (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    Contfrac                (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    evap                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow             (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                      ...
    mrsos                   (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    mrso                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    LAImean                 (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    snowtemp_weighted       (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    frac_snow               (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coast_and_river         (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         subgrid_halfdeg, no_irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Jan-26 16:07:13 GMT
    uuid:         a165ac95-4725-40e6-b8fb-7b8a30bd96f6
    LongName:     Ref config with new driver, WFDEI/v2 halfdeg, and VEGET_UPD...
    plot_color:   red

In [8]:
#open netcdf files for native routing on ORC grid
filename = '{}/eval_halfdeg/nat14/MO/nat14_*_1M_sechiba_history.nc'.format(dir)
name = 'interp_topo, no_irr'
natORC_noirr = format_routing_ORC(filename, name, spinupyears, color='orange')
natORC_noirr

<xarray.Dataset> Size: 5GB
Dimensions:                 (time: 3653, axis_nbounds: 2, lat: 36, lon: 38,
                             solay: 11, veget: 15, nobio: 1)
Coordinates:
  * lat                     (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                     (lon) float32 152B -14.75 -14.25 ... 3.25 3.75
  * solay                   (solay) float32 44B 0.0 0.001955 ... 0.999 2.0
  * veget                   (veget) float32 60B 1.0 2.0 3.0 ... 13.0 14.0 15.0
  * nobio                   (nobio) float32 4B 1.0
    time_centered           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 29kB 2003-01-01T12:00:00 .....
Dimensions without coordinates: axis_nbounds
Data variables: (12/68)
    time_centered_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds     (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    Areas                   (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    Contfrac                (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    evap                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow             (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                      ...
    mrsos                   (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    mrso                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    LAImean                 (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    snowtemp_weighted       (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    frac_snow               (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coast_and_river         (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, no_irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-01 13:47:52 GMT
    uuid:         9dd2c0bf-21ec-4a96-bd13-e125f9985bca
    LongName:     Ref config with new driver, WFDEI/v2 halfdeg, and VEGET_UPD...
    plot_color:   orange

In [9]:
#open netcdf files for native routing diag_routing_r (regridded to 38x36)
# filename = 'nat8.1/routing_native_grid_regridded/nat8.1_*.nc'
filename = '{}/eval_halfdeg/nat14/DA/nat14_*routing_r.nc'.format(dir)
name = 'interp_topo, no_irr'
natDIAGR_noirr = format_routing_native(filename, name, spinupyears, color='orange')
natDIAGR_noirr

<xarray.Dataset> Size: 566MB
Dimensions:                (time: 3653, axis_nbounds: 2, lat: 36, lon: 38,
                            basins: 200)
Coordinates:
  * lat                    (lat) float32 144B 44.75 44.25 43.75 ... 27.75 27.25
  * lon                    (lon) float32 152B -14.75 -14.25 -13.75 ... 3.25 3.75
  * basins                 (basins) |S255 51kB b'Amazon\x00                  ...
    time_instant           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                   (time) datetime64[ns] 29kB 2003-01-01T12:00:00 ......
    time_centered          (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/18)
    time_instant_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_centered_bounds   (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    flow_r                 (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    runoff                 (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_lakeinflow_r   (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                     ...
    hydrographs            (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrig_netereq_r        (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_irrigation_r   (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    basins_riverflow       (time, basins) float64 6MB dask.array<chunksize=(1, 200), meta=np.ndarray>
    routing_basins_area    (time, lat, lon) float64 40MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    routing_stream_diag_r  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, no_irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-01 13:47:53 GMT
    uuid:         d24203e9-83c1-4595-a282-a23274992ada
    plot_color:   orange

In [10]:
filename = '{}/eval_halfdeg/nat14/DA/nat14_*_routing.nc'.format(dir)
name = 'interp_topo, no_irr'
natDIAG_noirr = format_routing_nativeDIAG(filename, name, spinupyears, color='orange')
natDIAG_noirr

<xarray.Dataset> Size: 449MB
Dimensions:                (time: 3653, axis_nbounds: 2,
                            lat_domain_landpoints: 36,
                            lon_domain_landpoints: 38, lat: 36, lon: 38,
                            lat_routing_domain: 40, lon_routing_domain: 42)
Coordinates:
  * lat_domain_landpoints  (lat_domain_landpoints) float32 144B 27.25 ... 44.75
  * lon_domain_landpoints  (lon_domain_landpoints) float32 152B -14.75 ... 3.75
  * lat                    (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                    (lon) float32 152B -14.75 -14.25 -13.75 ... 3.25 3.75
  * lat_routing_domain     (lat_routing_domain) float32 160B 45.75 ... 26.25
  * lon_routing_domain     (lon_routing_domain) float32 168B -15.75 ... 4.75
    time_instant           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                   (time) datetime64[ns] 29kB 2003-01-01T12:00:00 ......
    time_centered          (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/14)
    time_instant_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_centered_bounds   (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    runoff                 (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    drainage               (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow            (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                     ...
    fastr                  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    slowr                  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    streamr                (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrigation             (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_basins_area    (time, lat_routing_domain, lon_routing_domain) float64 49MB dask.array<chunksize=(365, 40, 42), meta=np.ndarray>
    routing_stream_diag    (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, no_irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-01 13:47:53 GMT
    uuid:         ef8caca6-2569-40cc-8eef-2500882e3ead
    plot_color:   orange

## With irrigation

In [11]:
#open netcdf files for standard routing
filename = '{}/eval_halfdeg/nat16/nat16_sechiba_history_all.nc'.format(dir)
name = 'subgrid_halfdeg, irr'
std_irr = format_routing_ORC(filename, name, spinupyears, color='purple')
std_irr

<xarray.Dataset> Size: 5GB
Dimensions:                 (lat: 36, lon: 38, solay: 11, veget: 15, nobio: 1,
                             time: 3653, axis_nbounds: 2)
Coordinates:
  * lat                     (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                     (lon) float32 152B -14.75 -14.25 ... 3.25 3.75
  * solay                   (solay) float32 44B 0.0 0.001955 ... 0.999 2.0
  * veget                   (veget) float32 60B 1.0 2.0 3.0 ... 13.0 14.0 15.0
  * nobio                   (nobio) float32 4B 1.0
    time_centered           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 29kB 2003-01-01T12:00:00 .....
Dimensions without coordinates: axis_nbounds
Data variables: (12/94)
    time_centered_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds     (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    Areas                   (lat, lon) float32 5kB dask.array<chunksize=(36, 38), meta=np.ndarray>
    Contfrac                (lat, lon) float32 5kB dask.array<chunksize=(36, 38), meta=np.ndarray>
    evap                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow             (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                      ...
    Count_failure_stre      (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    root_deficit            (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    root_mc_fc              (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrigmap_dyn            (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    aei_sw                  (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coast_and_river         (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         subgrid_halfdeg, irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-15 10:57:49 GMT
    uuid:         9bd632b6-cbb8-4ee3-93df-569ce27b6425
    LongName:     Ref config with new driver, WFDEI/v2 halfdeg, and VEGET_UPD...
    history:      Thu Feb 15 12:31:44 2024: ncrcat nat16_20000101_20001231_1M...
    NCO:          netCDF Operators version 4.8.1 (Homepage = http://nco.sf.ne...
    plot_color:   purple

In [12]:
#native with irrig
filename = '{}/eval_halfdeg/nat15/MO/nat15*_sechiba_history.nc'.format(dir)
name = 'interp_topo, irr'
natORC_irr = format_routing_ORC(filename, name, spinupyears, color='green')
natORC_irr

<xarray.Dataset> Size: 6GB
Dimensions:                 (time: 3653, axis_nbounds: 2, lat: 36, lon: 38,
                             solay: 11, veget: 15, nobio: 1)
Coordinates:
  * lat                     (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                     (lon) float32 152B -14.75 -14.25 ... 3.25 3.75
  * solay                   (solay) float32 44B 0.0 0.001955 ... 0.999 2.0
  * veget                   (veget) float32 60B 1.0 2.0 3.0 ... 13.0 14.0 15.0
  * nobio                   (nobio) float32 4B 1.0
    time_centered           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 29kB 2003-01-01T12:00:00 .....
Dimensions without coordinates: axis_nbounds
Data variables: (12/86)
    time_centered_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds     (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    Areas                   (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    Contfrac                (time, lat, lon) float32 20MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    evap                    (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow             (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                      ...
    Count_failure_stre      (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    root_deficit            (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    root_mc_fc              (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrigmap_dyn            (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    aei_sw                  (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coast_and_river         (time, lat, lon) float32 20MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-06 14:43:04 GMT
    uuid:         3bea1703-977c-4ef9-881d-5d1f7d7590a3
    LongName:     Ref config with new driver, WFDEI/v2 halfdeg, and VEGET_UPD...
    plot_color:   green

In [13]:
#native with irrig
filename = '{}/eval_halfdeg/nat15/DA/nat15_*_routing_r.nc'.format(dir)
name = 'interp_topo, irr'
natDIAGR_irr = format_routing_native(filename, name, spinupyears, color='green')
natDIAGR_irr

<xarray.Dataset> Size: 566MB
Dimensions:                (time: 3653, axis_nbounds: 2, lat: 36, lon: 38,
                            basins: 200)
Coordinates:
  * lat                    (lat) float32 144B 44.75 44.25 43.75 ... 27.75 27.25
  * lon                    (lon) float32 152B -14.75 -14.25 -13.75 ... 3.25 3.75
  * basins                 (basins) |S255 51kB b'Amazon\x00                  ...
    time_instant           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                   (time) datetime64[ns] 29kB 2003-01-01T12:00:00 ......
    time_centered          (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/18)
    time_instant_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_centered_bounds   (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    flow_r                 (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    runoff                 (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_lakeinflow_r   (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                     ...
    hydrographs            (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrig_netereq_r        (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_irrigation_r   (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    basins_riverflow       (time, basins) float64 6MB dask.array<chunksize=(1, 200), meta=np.ndarray>
    routing_basins_area    (time, lat, lon) float64 40MB dask.array<chunksize=(365, 36, 38), meta=np.ndarray>
    routing_stream_diag_r  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-06 14:43:08 GMT
    uuid:         e417188b-d6df-4174-a3f6-932f37b66bc5
    plot_color:   green

In [14]:
#native with irrig
filename = '{}/eval_halfdeg/nat15/DA/nat15_*_routing.nc'.format(dir)
name = 'interp_topo, irr'
natDIAG_irr = format_routing_nativeDIAG(filename, name, spinupyears, color='green')
natDIAG_irr

<xarray.Dataset> Size: 449MB
Dimensions:                (time: 3653, axis_nbounds: 2,
                            lat_domain_landpoints: 36,
                            lon_domain_landpoints: 38, lat: 36, lon: 38,
                            lat_routing_domain: 40, lon_routing_domain: 42)
Coordinates:
  * lat_domain_landpoints  (lat_domain_landpoints) float32 144B 27.25 ... 44.75
  * lon_domain_landpoints  (lon_domain_landpoints) float32 152B -14.75 ... 3.75
  * lat                    (lat) float32 144B 27.25 27.75 28.25 ... 44.25 44.75
  * lon                    (lon) float32 152B -14.75 -14.25 -13.75 ... 3.25 3.75
  * lat_routing_domain     (lat_routing_domain) float32 160B 45.75 ... 26.25
  * lon_routing_domain     (lon_routing_domain) float32 168B -15.75 ... 4.75
    time_instant           (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
  * time                   (time) datetime64[ns] 29kB 2003-01-01T12:00:00 ......
    time_centered          (time) datetime64[ns] 29kB dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/14)
    time_instant_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds    (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_centered_bounds   (time, axis_nbounds) datetime64[ns] 58kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    runoff                 (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    drainage               (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    coastalflow            (time, lat_domain_landpoints, lon_domain_landpoints) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    ...                     ...
    fastr                  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    slowr                  (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    streamr                (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    irrigation             (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
    routing_basins_area    (time, lat_routing_domain, lon_routing_domain) float64 49MB dask.array<chunksize=(365, 40, 42), meta=np.ndarray>
    routing_stream_diag    (time, lat, lon) float64 40MB dask.array<chunksize=(1, 36, 38), meta=np.ndarray>
Attributes:
    name:         interp_topo, irr
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2024-Feb-06 14:43:08 GMT
    uuid:         fdfb88da-bd30-4881-b580-182b74c97ae6
    plot_color:   green

In [15]:
natORC_irr['irrigation'] = natDIAG_irr['irrigation'] / natORC_irr['Areas'] * 2 * 24
natORC_irr['irrigation'].attrs['long_name'] = 'irrigation'
natORC_irr['irrigation'].attrs['units'] = 'mm d⁻¹'

## Contfrac from other sim for tests

In [16]:
filename = '../../../contfrac_CRUJRA-halfdeg_global.nc'
contfrac = xr.open_dataset(filename)
#restrict to lon_min, lon_max, lat_min, lat_max
lon_min=-10
lon_max=4
lat_min=35.5
lat_max=44.5
contfrac = contfrac.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))
contfrac

<xarray.Dataset> Size: 4kB
Dimensions:   (lat: 18, lon: 28)
Coordinates:
  * lat       (lat) float32 72B 44.25 43.75 43.25 42.75 ... 36.75 36.25 35.75
  * lon       (lon) float32 112B -9.75 -9.25 -8.75 -8.25 ... 2.25 2.75 3.25 3.75
Data variables:
    Areas     (lat, lon) float32 2kB ...
    Contfrac  (lat, lon) float32 2kB ...
Attributes:
    name:         sechiba_history_1mo
    description:  Created by xios
    title:        Created by xios
    Conventions:  CF-1.6
    timeStamp:    2025-Mar-22 09:27:07 GMT
    uuid:         85debc55-ebae-465e-9950-618845ba8c95
    LongName:     ExpGlo_C_IntTopo_r8872_halfdeg
    history:      Fri Aug  1 10:34:08 2025: ncks -v Areas,Contfrac /ccc/store...
    NCO:          netCDF Operators version 5.0.1 (Homepage = http://nco.sf.ne...

# File subsampling

### Monthly files

In [17]:
#extract monthly mean from noirr datasets
# add 14 days to the time coordinate to sync with discharge observations
std_noirr_month= std_noirr.resample(time='MS').mean()
# std_noirr_month['time'] = std_noirr_month['time'] + np.timedelta64(14, 'D')

natORC_noirr_month= natORC_noirr.resample(time='MS').mean()
# natORC_noirr_month['time'] = natORC_noirr_month['time'] + np.timedelta64(14, 'D')

natDIAGR_noirr_month= natDIAGR_noirr.resample(time='MS').mean()
# natDIAGR_noirr_month['time'] = natDIAGR_noirr_month['time'] + np.timedelta64(14, 'D')

natDIAG_noirr_month= natDIAG_noirr.resample(time='MS').mean()
# natDIAG_noirr_month['time'] = natDIAG_noirr_month['time'] + np.timedelta64(14, 'D')

In [18]:
#extract monthly mean from irr datasets
# add 14 days to the time coordinate to sync with discharge observations
std_irr_month= std_irr.resample(time='MS').mean()
# std_irr_month['time'] = std_irr_month['time'] + np.timedelta64(14, 'D')

natORC_irr_month= natORC_irr.resample(time='MS').mean()
# natORC_irr_month['time'] = natORC_irr_month['time'] + np.timedelta64(14, 'D')

natDIAGR_irr_month= natDIAGR_irr.resample(time='MS').mean()
# natDIAGR_irr_month['time'] = natDIAGR_irr_month['time'] + np.timedelta64(14, 'D')

natDIAG_irr_month= natDIAG_irr.resample(time='MS').mean()
# natDIAG_irr_month['time'] = natDIAG_irr_month['time'] + np.timedelta64(14, 'D')

In [19]:
#make a version of std_noirr_month limited in lon lat
std_noirr_month_restricted_ave = std_noirr_month.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max)).mean(dim=['time'])
#broadcast contfrac temporally like std_noirr_restricted to time dimension
# contfrac_b = contfrac.broadcast_like(std_noirr_month_restricted)
#drop dims veget, nobio, solay
# contfrac_b = contfrac_b.drop_vars(['veget', 'nobio', 'solay'])
std_noirr_month_restricted_ave['fastr_corrected'] = std_noirr_month_restricted_ave['fastr'] / (contfrac['Contfrac'] ** 2)
std_noirr_month_restricted_ave['slowr_corrected'] = std_noirr_month_restricted_ave['slowr'] / (contfrac['Contfrac'] ** 2)

### Masks

In [20]:
#Iberian Peninsula mask
mask_IP = polygon_to_mask(std_noirr_month, iberian_peninsula)

In [21]:
# ip_std_noirr = std_noirr.where(mask_IP, drop=True)
# ip_natORC_noirr = natORC_noirr.where(mask_IP, drop=True)
# ip_natDIAGR_noirr = natDIAGR_noirr.where(mask_IP, drop=True)
# ip_natDIAG_noirr = natDIAG_noirr.where(mask_IP, drop=True)

ip_std_noirr_month = std_noirr_month.where(mask_IP, drop=True)
ip_natORC_noirr_month = natORC_noirr_month.where(mask_IP, drop=True)
ip_natDIAGR_noirr_month = natDIAGR_noirr_month.where(mask_IP, drop=True)
ip_natDIAG_noirr_month = natDIAG_noirr_month.where(mask_IP, drop=True)

In [22]:
#irr datasets
# ip_std_irr = std_irr.where(mask_IP, drop=True)
# ip_natORC_irr = natORC_irr.where(mask_IP, drop=True)
# ip_natDIAGR_irr = natDIAGR_irr.where(mask_IP, drop=True)
# ip_natDIAG_irr = natDIAG_irr.where(mask_IP, drop=True)

ip_std_irr_month = std_irr_month.where(mask_IP, drop=True)
ip_natORC_irr_month = natORC_irr_month.where(mask_IP, drop=True)
ip_natDIAGR_irr_month = natDIAGR_irr_month.where(mask_IP, drop=True)
ip_natDIAG_irr_month = natDIAG_irr_month.where(mask_IP, drop=True)

In [23]:
cont_mask = std_noirr_month.mean(dim='time')['coastalflow'] <= 0.0 
cont_mask2 = std_noirr_month.mean(dim='time')['riverflow'] <= 0.0

In [24]:
cont_ip_std_noirr_month = std_noirr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natORC_noirr_month = natORC_noirr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natDIAGR_noirr_month = natDIAGR_noirr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natDIAG_noirr_month = natDIAG_noirr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)

cont_ip_std_irr_month = std_irr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natORC_irr_month = natORC_irr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natDIAGR_irr_month = natDIAGR_irr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)
cont_ip_natDIAG_irr_month = natDIAG_irr_month.where(cont_mask, drop=False).where(cont_mask2, drop=False).where(mask_IP, drop=True)

In [25]:
coastalflow_mask = std_noirr_month.mean(dim='time')['coastalflow'] > 0.0
coastal_ip_std_noirr_month = std_noirr_month.where(coastalflow_mask, drop=False).where(mask_IP, drop=True)
coastal_ip_natORC_noirr_month = natORC_noirr_month.where(coastalflow_mask, drop=False).where(mask_IP, drop=True)

riverflow_mask = std_noirr_month.mean(dim='time')['riverflow'] > 0.0
outlets_ip_std_noirr_month = std_noirr_month.where(riverflow_mask, drop=False).where(mask_IP, drop=True)
outlets_ip_natORC_noirr_month = natORC_noirr_month.where(riverflow_mask, drop=False).where(mask_IP, drop=True)

In [26]:
coast_mask2 = (std_noirr_month.mean(dim='time')['coast_and_river'] > 0.0)

In [27]:
coastalflow_mask2 = natDIAGR_noirr_month.mean(dim='time')['coastalflow'] > 0.0
coastal_natDIAGR_noirr_month = natDIAGR_noirr_month.where(coastalflow_mask2, drop=False).where(mask_IP, drop=True)

riverflow_mask2 = natDIAGR_noirr_month.mean(dim='time')['riverflow'] > 0.0
outlets_natDIAGR_noirr_month = natDIAGR_noirr_month.where(riverflow_mask2, drop=False).where(mask_IP, drop=True)

# Tests

In [28]:
stop

NameError: name 'stop' is not defined

## Maps

In [ ]:
#Display map of var for ds
var='rain'
ds = contfrac
ds = coastal_ip_natORC_noirr_month
vmin=-15
vmax=100
vmin, vmax = None, None
cmap=reds

# title = 'Annual mean ' + ds[var].long_name + ' ({}) '.format(ds.attrs['name'])
title = var
# clabel= ds[var].name + ' (' + ds[var].units + ')'
clabel=None

map_ave(ds, var, cmap=cmap, vmin=vmin, vmax=vmax, title=title, clabel=clabel)

In [ ]:
#Display map of var for ds
var=''
ds = std_noirr_month_restricted_ave
# ds = ip_std_irr_month
vmin=0
vmax=20
# vmin, vmax = None, None
cmap=blues

# title = 'Annual mean ' + ds[var].long_name + ' ({}) '.format(ds.attrs['name'])
title = var
# clabel= ds[var].name + ' (' + ds[var].units + ')'
clabel=None

map_ave(ds, var, cmap=cmap, vmin=vmin, vmax=vmax, title=title, clabel=clabel)

In [ ]:
# Display map of var for 2 sims
var = 'fastr'
# max_val = None
max_val = 3
cmap = blues

ds1 = std_noirr_month_restricted_ave
ds1 = ip_std_noirr_month
# ds1 = std_noirr_month
# ds1 = ip_natDIAGR_noirr_month

# ds2 = ip_natDIAGR_noirr_month
# ds2 = cont_ip_natORC_noirr_month
# ds2 = ip_natDIAG_irr
ds2 = ip_natORC_noirr_month

if 'time' in ds1.dims:
    plotvar_std = ds1[var].mean(dim='time')
    plotvar_std=plotvar_std.where(plotvar_std>0)
else:
    plotvar_std = ds1[var]
    plotvar_std=plotvar_std.where(plotvar_std>0)
plotvar_nat = ds2[var].mean(dim='time')
plotvar_nat=plotvar_nat.where(plotvar_nat>0)

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(18, 4), subplot_kw={'projection': ccrs.PlateCarree()})
# fig.suptitle(ds1[var].long_name + ' average over the period (2003-2012, {})'.format(ds1[var].units))

# Plot for std dataset
axs[0].coastlines()
plotvar_std.plot(ax=axs[0], transform=ccrs.PlateCarree(), cmap=cmap, vmax=max_val)
if 'name' in ds1.attrs:
    axs[0].set_title(f'{ds1.name}')

# Plot for natORC dataset
axs[1].coastlines()
plotvar_nat.plot(ax=axs[1], transform=ccrs.PlateCarree(), cmap=cmap, vmax=max_val)
axs[1].set_title(f'{ds2.name}')

In [ ]:
#Diff map
var='streamr'
ds1=ip_natORC_noirr_month
ds2=ip_std_noirr_month
# ds2=ip_natD_noirr_month

vmax= 0.5
vmin=-0.5
# vmin, vmax = None, None
cmap=emb_neutral

title= ds1[var].attrs['long_name'] + ' difference (' + ds1.attrs['name'] + ' - ' + ds2.attrs['name'] + ')'
clabel= ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'


map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, title=title, clabel=clabel)
vmin, vmax=-100, 100
# map_rel_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap)

In [ ]:
#RMSE map
var='irrigation'
ds1 = ip_std_irr_month
ds2 = ip_natORC_irr_month
map_rmse_ave(ds1, ds2, var)

In [ ]:
#RMSE and rRMSE values for multiple variables
varlist=['streamr', 'fastr', 'slowr']
# varlist=['drainage', 'runoff']
ds1=ip_natORC_noirr
ds2=ip_std_noirr
#compute RMSE between ds1 and ds2
for var in varlist:
    rmse = np.sqrt(((ds1[var]-ds2[var])**2).mean(dim='time'))
    # rel_rmse = rmse/ds2[var].mean(dim='time')
    #output the value of the rmse after spatial averaging over all grid points
    print(var + ' RMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rmse.mean(dim=['lon','lat']).values)))
    # print(var + ' rRMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rel_rmse.mean(dim=['lon','lat']).values)))

## Time series

In [ ]:
#Average time series
var='mrsos'
year_min=2000
year_max=2012

ds1 = ip_std_noirr_month
ds2 = ip_natORC_noirr_month
ds3 = ip_std_irr_month
ds4 = ip_natORC_irr_month
ds_list = [ds1, ds2, ds3, ds4]
# ds_list = [ds3, ds4]

time_series_ave(ds_list, var, 
                year_min=year_min, year_max=year_max,
                ds_colors=True
                )

In [ ]:
var='evap'
year_min=2003
year_max=2012

ds1 = ip_std_noirr_month
ds2 = ip_natORC_noirr_month
ds3 = ip_std_irr_month
ds4 = ip_natORC_irr_month
ds_list=[ds1, ds2, ds3, ds4]
ds_list = [ds3, ds4]


seasonal_cycle_ave(ds_list, var, 
                   year_min=year_min, year_max=year_max,
                   ds_colors=True
                   )

## Hydrographs

In [ ]:
filename = '../../../obs/streamflow/GRDC-Monthly_Spain.nc'
obs_routing_grdc = xr.open_dataset(filename)
obs_routing_grdc

In [ ]:
filename = '../../../obs/streamflow/GRDC_Monthly_Jan20_v1_ES.nc'
obs_routing_polcher = xr.open_dataset(filename)
rename_dict = {'hydrographs': 'runoff_mean',
               'number': 'id',}
obs_routing_polcher = obs_routing_polcher.rename(rename_dict)
#shift time coordinate to match the sims and grdc obs
obs_routing_polcher['time'] = obs_routing_polcher['time'] - np.timedelta64(14, 'D')
obs_routing_polcher

In [ ]:
proper_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216520: {'name': 'Villarubia',         'river': 'Guadiana',        'lat_grid': 39.125,     'lon_grid': -3.59073,   'station_nb': 14},      
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
    6217700: {'name': 'Pinos Puente',       'river': 'Frailes',         'lat_grid': 37.27499,   'lon_grid': -3.75832,   'station_nb': 18},
}

#keeping only 5 representative stations for larger rivers, for simple figure
representative_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    # 6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
}

stations_calib_v0 = {
    6298249: {'name': 'Zaragoza',  'river': 'Ebro', 'lon_grid':-0.8749926686286926, 'lat_grid':41.65833282470703, 'year_min':2003, 'year_max':2012, 'station_nb':1},
    6298992: {'name': 'Albarreal De Tajo',  'river': 'Tagus', 'lon_grid':-4.17499303817749, 'lat_grid':39.891666412353516, 'year_min':2003, 'year_max':2012 , 'station_nb':2},
    6298481: {'name': 'San Miguel del Pino', 'river': 'Douro', 'lon_grid':-4.92499303817749, 'lat_grid':41.508331298828125, 'year_min':2003, 'year_max':2012, 'station_nb':3},
    6298259:{'name': 'Badajoz', 'river': 'Guadiana', 'lat_grid': 38.85833358764648, 'lon_grid': -7.008326530456543, 'last_record': '2013-09-15', 'year_min':2003, 'year_max':2012, 'station_nb':4},
                }

stations_calib_v1 = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
}

In [ ]:
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(20,10))
# fig, axes= plt.subplots(5,3, figsize=(25,20))

axes= axes.flatten()
ds_list=[std_noirr_month, natDIAGR_noirr_month, std_irr_month, natDIAGR_irr_month]
# ds_list = [std_noirr_month]
for i, (station_id, station_data) in enumerate(stations_dict.items()):
    # title_letter=letters[i]
    ts_station(obs_ds,axes[i], station_id, polcher_ds=polcher_ds, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012)
    ts_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, polcher_ds=polcher_ds, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, plot_all_sim=False)


In [ ]:
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(18,8))
# fig, axes= plt.subplots(5,3, figsize=(25,20))

axes= axes.flatten()
ds_list=[std_noirr_month, natDIAGR_noirr_month, std_irr_month, natDIAGR_irr_month]
ds_list= [std_noirr_month]
# for i, (station_id, station_data) in enumerate(representative_stations_dict.items()):
for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds, title_letter=title_letter)

# Figures

## 3.1 : map of used stations and characteristics

In [ ]:
savefig=True
stations_map_dict(stations_calib_v0, river_cond=None, legend=True,
                            dams_df=None, dam_nb=252, 
                            title=None,
                            extent=[-10, 2.5, 36, 44]
                            )

if savefig:
    plt.savefig('figures/river_discharge/halfdeg_4stations_map.png', bbox_inches='tight')

In [ ]:
stations_ds=obs_routing_polcher
polcher_ds = True
for station_id, station_data in stations_calib_v0.items():
    mask = (stations_ds['id'] == station_id)
    station = stations_ds.sel(stations=mask)
    coverage = percent_valid_coverage(station, 2003, 2012, time_var='time', data_var='runoff_mean')
    print(f"Station Name: {station_data['name']}, number: {station_data['station_nb']} Coverage: {coverage}%")


## 3.2 : 12 maps (+ RMSD in case)

In [ ]:
savefig=True
varlist = ['fastr', 'slowr', 'streamr']
# varlist = ['streamr']
ds1 = ip_natORC_noirr_month
ds2 = ip_std_noirr_month
vmins = [0, 0, 0]
# vmaxs = [4, 110, 20]
vmaxs = [4, 110, 20]
vmaxs_diff=[2, 100, 5]

cmap_value=blues
cmap_diff=emb_neutralW
cmap_rmse=redsW

title='off'

for i in range(len(varlist)):
    var = varlist[i]
    vmin = vmins[i]
    vmax = vmaxs[i]
    clabel= ds1[var].attrs['long_name'] + ' (kg m⁻²)'

    #both datasets
    title= ds1[var].attrs['long_name'] + ' (' + ds1.attrs['name'] + ')'
    title='off'
    map_ave(ds1, var, cmap=cmap_value, vmin=vmin, vmax=vmax, title=title, clabel=clabel)
    if savefig:
        plt.savefig('figures/chap3/maps/{}_interp.png'.format(var), bbox_inches='tight')

    # title= ds1[var].attrs['long_name'] + ' ( ' + ds2.attrs['name'] + ')'
    map_ave(ds2, var, cmap=cmap_value, vmin=vmin, vmax=vmax, title=title, clabel=clabel)
    if savefig:
        plt.savefig('figures/chap3/maps/{}_subgrid.png'.format(var), bbox_inches='tight')

    #diff ds1-ds2
    vmin = - vmaxs_diff[i]
    vmax = vmaxs_diff[i]
    # title= ds1[var].attrs['long_name'] + ' difference (interp_topo - subgrid_halfdeg)'
    map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap_diff, title=title, clabel=clabel)
    if savefig:
        plt.savefig('figures/chap3/maps/{}_diff.png'.format(var), bbox_inches='tight')

    #RMSD ds1-ds2
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel= f'{ds1[var].attrs['long_name']} RMSD (kg m⁻²)'
    map_rmse_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, clabel=clabel, title='off')
    if savefig:
        plt.savefig('figures/chap3/maps/{}_rmse.png'.format(var), bbox_inches='tight')

In [ ]:
savefig=True
var='hydrographs'
ds1 = ip_natDIAGR_noirr_month
ds2 = ip_std_noirr_month
vmin=0
vmax=600
diff_minmax=100
# diff_minmax=600

cmap_value=blues
cmap_diff=emb_neutralW
cmap_rmse=redsW

clabel = 'River discharge (m³ s⁻¹)'
title='off'

#both datasets
# title= ds1[var].attrs['long_name'] + ' ( ' + ds1.attrs['name'] + ')'
# map_ave(ds1, var, cmap=cmap_value, vmin=vmin, vmax=vmax, title=title, clabel=clabel)
# if savefig:
#     plt.savefig('figures/chap3/maps/{}_interp.png'.format(var), bbox_inches='tight')

# # title= ds1[var].attrs['long_name'] + ' ( ' + ds2.attrs['name'] + ')'
# map_ave(ds2, var, cmap=cmap_value, vmin=vmin, vmax=vmax, title=title, clabel=clabel)
# if savefig:
#     plt.savefig('figures/chap3/maps/{}_subgrid.png'.format(var), bbox_inches='tight')

# #diff ds1-ds2
# # title= ds1[var].attrs['long_name'] + ' difference (interp_topo - subgrid_halfdeg)'
# map_diff_ave(ds1, ds2, var, vmin=-diff_minmax, vmax=diff_minmax, cmap=cmap_diff, title=title, clabel=clabel)
# if savefig:
#     plt.savefig('figures/chap3/maps/{}_diff.png'.format(var), bbox_inches='tight')

#RMSD ds1-ds2
clabel= 'River discharge RMSD (m³ s⁻¹)'
map_rmse_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, clabel=clabel, cmap=reds, title='off')
if savefig:
    plt.savefig('figures/chap3/maps/{}_rmse.png'.format(var), bbox_inches='tight')

## 3.3 : Time series / seasonnal cycle on average over Peninsula

In [ ]:
savefig=True
plot_TS=True
plot_SC=True

varlist=['slowr', 'fastr', 'streamr']
# varlist=['slowr']
year_min=2003
year_max=2012
vmins = [0, 0, 0]
vmaxs = [50, 3, 6]
vmaxs_SC = [20, 1, 3]

ds1 = ip_std_noirr_month
ds2 = ip_natORC_noirr_month
ds5= ip_std_irr_month
ds6 = ip_natORC_irr_month

ds_list, cont = [ds1, ds2], False
ds_list, cont = [ds1, ds2, ds5, ds6], False
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs[varlist.index(var)]
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_time_series.png'.format(var), bbox_inches='tight')

    #Seasonal cycle
    if plot_SC:
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs_SC[varlist.index(var)]
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_seasonal_cycle.png'.format(var), bbox_inches='tight')

In [ ]:
savefig=True
plot_TS=True
plot_SC=True

varlist=['hydrographs']
year_min=2003
year_max=2012
vmins = [0, 0, 0]
vmaxs = [50, 3, 6]
vmaxs_SC = [20, 1, 3]

ds1 = ip_std_noirr_month
ds2 = ip_natDIAGR_noirr_month
ds5= ip_std_irr_month
ds6 = ip_natDIAGR_irr_month

ds_list, cont = [ds1, ds2], False
ds_list, cont = [ds1, ds2, ds5, ds6], False
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs[varlist.index(var)]
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        # vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_time_series.png'.format(var), bbox_inches='tight')

    #Seasonal cycle
    if plot_SC:
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs_SC[varlist.index(var)]
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            # vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_seasonal_cycle.png'.format(var), bbox_inches='tight')

### Appendix : same without coastal grid cells

In [ ]:
savefig=True
plot_TS=True
plot_SC=True

varlist=['slowr', 'fastr', 'streamr']
# varlist=['slowr']
year_min=2003
year_max=2012
vmins = [0, 0, 0]
vmaxs = [50, 3, 6]
vmaxs_SC = [20, 1, 3]

ds1 = cont_ip_std_noirr_month
ds2 = cont_ip_natORC_noirr_month
ds3 = cont_ip_std_irr_month
ds4 = cont_ip_natORC_irr_month  

ds_list, cont = [ds1, ds2], True
ds_list, cont = [ds1, ds2, ds3, ds4], True
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs[varlist.index(var)]
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            if cont:
                plt.savefig('figures/chap3/time_series/cont_{}_time_series.png'.format(var), bbox_inches='tight')


    #Seasonal cycle
    if plot_SC:
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs_SC[varlist.index(var)]
        ylabel = ds1[var].attrs['long_name'] + ' (kg m⁻²)'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            if cont:
                plt.savefig('figures/chap3/time_series/cont_{}_seasonal_cycle.png'.format(var), bbox_inches='tight')

In [ ]:
savefig=True
plot_TS=True
plot_SC=True

varlist=['hydrographs']
year_min=2003
year_max=2012
vmins = [0, 0, 0]
vmaxs = [50, 3, 6]
vmaxs_SC = [20, 1, 3]

ds1 = cont_ip_std_noirr_month
ds2 = cont_ip_natDIAGR_noirr_month
ds5= cont_ip_std_irr_month
ds6 = cont_ip_natDIAGR_irr_month

# ds_list, cont = [ds1, ds2], True
ds_list, cont = [ds1, ds2, ds5, ds6], True
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs[varlist.index(var)]
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        # vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_cont_time_series.png'.format(var), bbox_inches='tight')

    #Seasonal cycle
    if plot_SC:
        vmin = vmins[varlist.index(var)]
        vmax = vmaxs_SC[varlist.index(var)]
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            # vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_cont_seasonal_cycle.png'.format(var), bbox_inches='tight')

## 3.4 Irrigation maps and TS+SC

In [ ]:
#maps
savefig=True
var='irrigation'
ds1 = ip_natORC_irr_month
ds2 = ip_std_irr_month

map_min, map_max = 0, 0.6
year_min=2003
year_max=2012

map_ave(ds1, var, cmap=wet, vmin=map_min, vmax=map_max, 
        title='off',
        clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig(f'figures/chap3/maps/{var}_interp.png', bbox_inches='tight')

map_ave(ds2, var, cmap=wet, vmin=map_min, vmax=map_max,
        title='off',
        clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig(f'figures/chap3/maps/{var}_subgrid.png', bbox_inches='tight')  

map_diff_ave(ds1, ds2, var, vmin=-0.3, vmax=0.3, cmap=emb_neutralW,
            title='off',
            # n_ticks=10,
            clabel='Irrigation difference (mm d⁻¹)')
if savefig:
    plt.savefig(f'figures/chap3/maps/{var}_diff.png', bbox_inches='tight')

rmse = np.sqrt(((ds1[var]-ds2[var])**2).mean(dim='time'))
rmse = rmse.where(rmse>0)
map_plotvar(rmse, cmap=redsW, vmin=0, vmax=0.6, 
            title='off',
            clabel='Irrigation RMSD (mm d⁻¹)')
if savefig:
    plt.savefig(f'figures/chap3/maps/{var}_rmse.png', bbox_inches='tight')

map_rel_diff_ave(ds1, ds2, var, vmin=-100, vmax=100, cmap=emb_neutralW,
            title='off',
            # n_ticks=10,
            clabel='Irrigation relative difference (%)')
if savefig:
    plt.savefig(f'figures/chap3/maps/{var}_reldiff.png', bbox_inches='tight')

In [ ]:
savefig=True
var='irrigation'
ds1 = ip_std_irr_month
ds2 = ip_natORC_irr_month
ds_list=[ds1, ds2]

year_min=2003
year_max=2012

ylabel='Irrigation (mm d⁻¹)'

time_series_ave(ds_list, var, year_min=year_min, year_max=year_max, ds_colors=True, ylabel=ylabel, title='off')
if savefig:
    plt.savefig(f'figures/chap3/time_series/{var}_time_series.png', bbox_inches='tight')

seasonal_cycle_ave(ds_list, var, year_min=year_min, year_max=year_max,  ylabel=ylabel, 
                   ds_colors=True, title='off')
if savefig:
    plt.savefig(f'figures/chap3/time_series/{var}_seasonal_cycle.png', bbox_inches='tight')

In [ ]:
#non coastal points only
savefig=False
var='irrigation'
ds1 = cont_ip_std_irr_month
ds2 = cont_ip_natORC_irr_month
ds_list=[ds1, ds2]

year_min=2003
year_max=2012

ylabel='Irrigation (mm d⁻¹)'

time_series_ave(ds_list, var, year_min=year_min, year_max=year_max, ds_colors=True, ylabel=ylabel, title='off')
if savefig:
    plt.savefig(f'figures/time_series/{var}_time_series.png', bbox_inches='tight')

seasonal_cycle_ave(ds_list, var, year_min=year_min, year_max=year_max,  ylabel=ylabel, 
                   ds_colors=True, title='off')
if savefig:
    plt.savefig(f'figures/time_series/{var}_seasonal_cycle.png', bbox_inches='tight')

## 3.5 : débits sur 4 stations, subgrid et interp, no_irr et irr

In [ ]:
savefig=True

In [ ]:
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(18,8))

axes= axes.flatten()
ds_list=[std_noirr_month, natDIAGR_noirr_month, std_irr_month, natDIAGR_irr_month]
for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    ts_station(obs_ds,axes[i], station_id, polcher_ds=polcher_ds, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012)
    ts_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, polcher_ds=polcher_ds, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, plot_all_sim=False, title_letter=title_letter)
if savefig:
    plt.savefig('figures/river_discharge/halfdeg_4stations_TS.png', bbox_inches='tight')

In [ ]:
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(18,8))

axes= axes.flatten()
ds_list=[std_noirr_month, natDIAGR_noirr_month, std_irr_month, natDIAGR_irr_month]
for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds, title_letter=title_letter)
if savefig:
    plt.savefig('figures/river_discharge/halfdeg_4stations_SC.png', bbox_inches='tight')